In [1]:
import requests
import pandas as pd

def fetch_bithumb_candles(pair: str = 'BTC_KRW', interval: str = '1h') -> pd.DataFrame:
    """
    Bithumb 공개 API에서 지정된 종목(pair)과 차트 간격(interval) 캔들 데이터를 가져와
    pandas DataFrame으로 반환합니다.
    """
    url = f'https://api.bithumb.com/public/candlestick/{pair}/{interval}'
    resp = requests.get(url)
    resp.raise_for_status()

    result = resp.json()
    if result.get('status') != '0000':
        raise RuntimeError(f"API error, status: {result.get('status')}")

    # data 필드에 [timestamp, open, close, high, low, volume] 리스트가 들어있음
    candles = result['data']

    # API가 최신순으로 반환하는 경우 시간순으로 뒤집기
    candles.reverse()

    # DataFrame 생성
    df = pd.DataFrame(candles, columns=[
        'timestamp',  # UNIX 시간(ms)
        'open',       # 시가
        'close',      # 종가
        'high',       # 고가
        'low',        # 저가
        'volume'      # 거래량
    ])

    # timestamp를 datetime으로 변환 (단위 ms -> 초 단위라면 unit='s' 로 변경)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    return df

if __name__ == '__main__':
    # 1) 데이터 가져오기
    df = fetch_bithumb_candles('BTC_KRW', '1h')

    # 2) DataFrame 확인
    print(df.head())

    # 3) CSV로 저장
    output_path = 'btc_1h_candlestick.csv'
    df.to_csv(output_path)
    print(f"Saved to {output_path}")


                          open      close       high        low       volume
timestamp                                                                   
2025-05-30 06:00:00  148341000  148332000  148441000  148315000   5.27757559
2025-05-30 05:00:00  148416000  148402000  148600000  148319000  16.85723878
2025-05-30 04:00:00  148670000  148401000  148684000  148267000  16.65593647
2025-05-30 03:00:00  148502000  148684000  149099000  148439000  29.68742287
2025-05-30 02:00:00  147996000  148502000  148751000  147990000   24.5188414
Saved to btc_1h_candlestick.csv
